# Sequence classification model for IMDB Sentiment Analysis
(c) Deniz Yuret, 2018

* Objectives: Learn the structure of the IMDB dataset and train a simple RNN model.
* Prerequisites: RNN models (06.rnn.ipynb), param, GRU, nll, minibatch, accuracy, Adam, train!
* Knet: dir (used by imdb.jl)

In [ ]:
using Pkg
for p in ("Knet","ProgressMeter")
    haskey(Pkg.installed(),p) || Pkg.add(p)
end

In [ ]:
EPOCHS=3          # Number of training epochs
BATCHSIZE=64      # Number of instances in a minibatch
EMBEDSIZE=125     # Word embedding size
NUMHIDDEN=100     # Hidden layer size
MAXLEN=150        # maximum size of the word sequence, pad shorter sequences, truncate longer ones
VOCABSIZE=30000   # maximum vocabulary size, keep the most frequent 30K, map the rest to UNK token
NUMCLASS=2        # number of output classes
DROPOUT=0.0       # Dropout rate
LR=0.001          # Learning rate
BETA_1=0.9        # Adam optimization parameter
BETA_2=0.999      # Adam optimization parameter
EPS=1e-08         # Adam optimization parameter

## Load and view data

In [ ]:
using Knet: Knet
ENV["COLUMNS"]=92                     # column width for array printing
include(Knet.dir("data","imdb.jl"))   # defines imdb loader

In [ ]:
@doc imdb

In [ ]:
@time (xtrn,ytrn,xtst,ytst,imdbdict)=imdb(maxlen=MAXLEN,maxval=VOCABSIZE);

In [ ]:
summary.((xtrn,ytrn,xtst,ytst,imdbdict))

In [ ]:
# Words are encoded with integers
rand(xtrn)'

In [ ]:
# Each word sequence is padded or truncated to length 150
length.(xtrn)'

In [ ]:
# Define a function that can print the actual words:
imdbvocab = Array{String}(undef,length(imdbdict))
for (k,v) in imdbdict; imdbvocab[v]=k; end
imdbvocab[VOCABSIZE-2:VOCABSIZE] = ["<unk>","<s>","<pad>"]
function reviewstring(x,y=0)
    x = x[x.!=VOCABSIZE] # remove pads
    """$(("Sample","Negative","Positive")[y+1]) review:\n$(join(imdbvocab[x]," "))"""
end

In [ ]:
# Hit Ctrl-Enter to see random reviews:
r = rand(1:length(xtrn))
println(reviewstring(xtrn[r],ytrn[r]))

In [ ]:
# Here are the labels: 1=negative, 2=positive
ytrn'

## Define the model

In [ ]:
using Knet: param, dropout, RNN

In [ ]:
struct SequenceClassifier; input; rnn; output; end

In [ ]:
SequenceClassifier(input::Int, embed::Int, hidden::Int, output::Int) =
    SequenceClassifier(param(embed,input), RNN(embed,hidden,rnnType=:gru), param(output,hidden))

In [ ]:
function (sc::SequenceClassifier)(input; pdrop=0)
    embed = sc.input[:, permutedims(hcat(input...))]
    embed = dropout(embed,pdrop)
    hidden = sc.rnn(embed)
    hidden = dropout(hidden,pdrop)
    return sc.output * hidden[:,:,end]
end

## Experiment

In [ ]:
using Knet: minibatch
dtrn = minibatch(xtrn,ytrn,BATCHSIZE;shuffle=true)
dtst = minibatch(xtst,ytst,BATCHSIZE)
length.((dtrn,dtst))

In [ ]:
# For running experiments
using Knet: train!, Adam, AutoGrad
import ProgressMeter

function trainresults(file,model)
    if (print("Train from scratch? ");readline()[1]=='y')
        updates = 0; prog = ProgressMeter.Progress(EPOCHS * length(dtrn))
        function callback(J)
            ProgressMeter.update!(prog, updates)
            return (updates += 1) <= prog.n
        end
        opt = Adam(lr=LR, beta1=BETA_1, beta2=BETA_2, eps=EPS)
        train!(model, dtrn; callback=callback, optimizer=opt, pdrop=DROPOUT)
        Knet.gc()
        Knet.save(file,"model",model)
    else
        isfile(file) || download("http://people.csail.mit.edu/deniz/models/tutorial/$file",file)
        model = Knet.load(file,"model")
    end
    return model
end

In [ ]:
using Knet: nll, accuracy
model = SequenceClassifier(VOCABSIZE,EMBEDSIZE,NUMHIDDEN,NUMCLASS)
# nll(model,dtrn), nll(model,dtst), accuracy(model,dtrn), accuracy(model,dtst)
# (0.6932078f0, 0.69321173f0, 0.4817708333333333, 0.48233173076923075)

In [ ]:
model = trainresults("imdbmodel.jld2",model); # 22s

In [ ]:
# nll(model,dtrn), nll(model,dtst), accuracy(model,dtrn), accuracy(model,dtst)
# (0.04928492f0, 0.39949256f0, 0.9883413461538462, 0.8558894230769231)

## Playground

In [ ]:
predictstring(x)="\nPrediction: " * ("Negative","Positive")[argmax(Array(vec(model([x]))))]
UNK = VOCABSIZE-2
str2ids(s::String)=[(i=get(imdbdict,w,UNK); i>=UNK ? UNK : i) for w in split(lowercase(s))]

In [ ]:
# Here we can see predictions for random reviews from the test set; hit Ctrl-Enter to sample:
r = rand(1:length(xtst))
println(reviewstring(xtst[r],ytst[r]))
println(predictstring(xtst[r]))

In [ ]:
# Here the user can enter their own reviews and classify them:
println(predictstring(str2ids(readline(stdin))))